#  De beaux graphiques avec python: mise en pratique


In [ ]:
%%R
library(knitr)  
library(reticulate)  
knitr::knit_engines$set(python = reticulate::eng_python)
knitr::opts_chunk$set(fig.path = "")
knitr::opts_chunk$set(eval = TRUE, echo = FALSE)

# Hook from Maelle Salmon: https://ropensci.org/technotes/2020/04/23/rmd-learnings/
knitr::knit_hooks$set(
  plot = function(x, options) {
    hugoopts <- options$hugoopts
    paste0(
      "{", "{<figure src=", # the original code is simpler
      # but here I need to escape the shortcode!
      '"', x, '" ',
      if (!is.null(hugoopts)) {
        glue::glue_collapse(
          glue::glue('{names(hugoopts)}="{hugoopts}"'),
          sep = " "
        )
      },
      ">}}\n"
    )
  }
)


In [ ]:
import os
os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = 'C:/Users/W3CRK9/AppData/Local/r-miniconda/envs/r-reticulate/Library/plugins/platforms'
os.environ["PROJ_LIB"] = r'C:\Users\W3CRK9\AppData\Local\r-miniconda\pkgs\proj4-4.9.3-hfa6e2cd_9\Library\share'
os.environ['GDAL_DATA'] = r"C:\Users\W3CRK9\AppData\Local\r-miniconda\envs\r-reticulate\Library\share\gdal"

La pratique de la visualisation se fera, dans ce cours, en répliquant des graphiques qu'on peut trouver sur
la page de l'*open-data* de la ville de Paris 
[ici](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/information/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name).

Ce TP vise à initier:

* Aux packages [matplotlib](https://matplotlib.org/) et
[seaborn](https://seaborn.pydata.org/) pour la construction de graphiques figés
* Au package [plotly](https://plotly.com/python/) pour les graphiques 
dynamiques, au format HTML

Nous verrons par la suite la manière de construire des cartes facilement avec
des formats équivalents.

Un sous-ensemble des données de [paris open data](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/information/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name) a été mis à disposition
sur <a href="https://github.com/linogaliana/python-datascientist/blob/master/data/bike.csv" class="github"><i class="fab fa-github"></i></a> pour faciliter l'import (élimination des colonnes
qui ne nous servirons pas mais ralentissent l'import)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Premier graphique avec l'API `matplotlib` de `pandas`

**Exercice 1: Importer les données et produire un premier graphique**

1.. Importer les données `velib`. Attention, il s'agit de données
compressées au format `gzip`, il faut donc utiliser l'option `compression`
2. En premier lieu, sans se préoccuper des éléments de style ni des labels des
graphiques, reproduire les deux premiers graphiques de la
[page d'analyse des données](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/dataviz/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name):
*Les 10 compteurs avec la moyenne horaire la plus élevée* et *Les 10 compteurs ayant comptabilisés le plus de vélos*.

*Conseil*: Pour obtenir un graphique ordonné du plus grand au plus petit, il faut avoir les données ordonnées du plus petit au
plus grand. C'est bizarre mais c'est comme ça...


In [ ]:
import pandas as pd
df = pd.read_csv('./data/bike.csv', compression = 'gzip')

df1 = df.groupby('Nom du compteur').agg({'Comptage horaire': "mean"}).sort_values('Comptage horaire', ascending = False).head(10)
df2 = df.groupby('Nom du compteur').agg({'Comptage horaire': "sum"}).sort_values('Comptage horaire', ascending = False).head(10)

In [ ]:
df1.sort_values('Comptage horaire').plot(kind = "barh", figsize = (25,5), color = 'red')

In [ ]:
df2.sort_values('Comptage horaire').plot(kind = "barh", figsize = (25,5), color = 'green')

On peut remarquer plusieurs éléments problématiques (par exemple les labels) mais
aussi des éléments ne correspondant pas (les titres des axes, etc.) ou 
manquants (le nom du graphique...)

Comme les graphiques produits par `pandas` suivent la logique très flexible
de `matplotlib`, il est possible de les customiser. Cependant, c'est
souvent beaucoup de travail et il peut être préférable de directement
utiliser *seaborn*, qui offre quelques arguments prêts à l'emploi

## Utiliser directement `seaborn`

Vous pouvez repartir des deux dataframes précédents. On va suppose qu'ils se
nomment `df1` et `df2`. Réinitialiser l'index pour avoir une colonne 
*'Nom du compteur'*

In [ ]:
df1 = df1.reset_index()
df2 = df2.reset_index()

**Exercice 2: un peu de style**

Il y a plusieurs manières de faire un *bar* plot en `seaborn`. La plus flexible,
c'est-à-dire celle qui permet le mieux d'interagir avec `matplotlib` est
`catplot`
 
1. Refaire le graphique précédent avec la fonction adéquate de `seaborn`. Pour
contrôler la taille du graphique vous pouvez utiliser les arguments `height` et
`aspect`
2. Ajouter les titres des axes et le titre du graphique pour le premier graphique

In [ ]:
# Q1 + Q2
g = sns.catplot(x='Comptage horaire', y='Nom du compteur', data=df1, kind = "bar", height = 5, aspect = 4, color = "red")
g.set_axis_labels('Moyenne du comptage par heure sur la période sélectionnée', 'Nom du compteur')
plt.title('Les 10 compteurs avec la moyenne horaire la plus élevée')
plt.show()

In [ ]:
# ALTERNATIVE Q1 + Q2
plt.figure(figsize=(20,5))
sns.barplot(x='Comptage horaire', y='Nom du compteur', data=df1, color = "red")
plt.xlabel('Moyenne du comptage par heure sur la période sélectionnée', size = 24)
plt.xlabel('Nom du compteur', size = 24)
plt.title('Les 10 compteurs avec la moyenne horaire la plus élevée', size=24)
plt.show()

Si vous désirez colorer en rouge l'axe des `x`, vous pouvez pré-définir un
style avec `sns.set_style("ticks", {"xtick.color": "red"})`

In [ ]:
sns.set_style("ticks", {"xtick.color": "red"})
g = sns.catplot(x='Comptage horaire', y='Nom du compteur', data=df1, kind = "bar", height = 10, aspect = 2, color = "red")
g.set_axis_labels('Moyenne du comptage par heure sur la période sélectionnée', 'Nom du compteur')
plt.title('Les 10 compteurs avec la moyenne horaire la plus élevée')
plt.show()

**Exercice 3: refaire les graphiques**

Certaines opérations vont nécessiter un peu d'agilité dans la gestion des dates.
Avant cela, il faut créer une variable temporelle (`datetime`). Pour cela, vous
pouvez faire:

In [ ]:
df['timestamp'] = pd.to_datetime(df['Date et heure de comptage'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')

.to_period('M')

1. Refaire le graphique *Les 10 compteurs ayant comptabilisés le plus de vélos *
2. Refaire le graphique *Moyenne mensuelle des comptages vélos *. Pour cela,
vous pouvez utiliser `.dt.to_period('M')` pour transformer un timestamp 
en variable de mois
3. Refaire le graphique *Moyenne journalière des comptages vélos* (créer d'abord
une variable de jour avec `.dt.day`)
4. Refaire le graphique *Comptages vélo au cours des 7 derniers jours * (de l'échantillon)

In [ ]:
# Q1
sns.set_style("ticks", {"xtick.color": "green"})
g = sns.catplot(x='Comptage horaire', y='Nom du compteur', data=df2, kind = "bar", height = 5, aspect = 4, color = "green")
g.set_axis_labels('La somme des vélos comptabilisés sur la période sélectionnée', 'Nom du compteur')
plt.title('Les 10 compteurs ayant comptabilisés le plus de vélos')
plt.show()

In [ ]:
# Q2
df['month'] = df.timestamp.dt.to_period('M')
sns.set_style("whitegrid")
df3 = df.groupby('month').agg({'Comptage horaire':'mean'}).reset_index()
g = sns.catplot(x='month', y='Comptage horaire', data=df3, kind = "bar", height = 5, aspect = 4, color = "yellow")
g.set_axis_labels('Date et heure de comptage', 'Moyenne mensuelle du comptage par heure sur la période sélectionnée')
plt.title('Moyenne mensuelle des comptages vélos')
plt.show()

In [ ]:
# Q3
df['day'] = df.timestamp.dt.date

df4 = df.groupby('day').agg({'Comptage horaire':'mean'}).reset_index()
sns.lineplot(x='day', y='Comptage horaire', data=df4, color = "magenta")

# TO DO: fill sous la courbe

In [ ]:
# Q4
df['Date'] = pd.to_datetime(df['day'])
df['NewDate'] = pd.to_datetime(df.Date) - pd.to_timedelta(7, unit="D")
df5 = df[df.day >= (max(df['NewDate']))]
df5 = df5.groupby('Date').agg({'Comptage horaire': 'sum'})
g = sns.catplot(y='Comptage horaire', x='Date', data=df5, kind = "bar", height = 10, aspect = 2, color = "lightblue")

## Des graphiques dynamiques avec `Plotly`

Le package `Plotly` est une surcouche à la librairie Javascript 
`Plotly.js` qui permet de créer et manipuler des objets graphiques de manière
très flexible afin de produire des objets réactifs sans avoir à recourir
à Javascript.

Le point d'entrée recommandé est le module `Plotly Express`
([documentation ici](https://plotly.com/python/plotly-express/)) qui offre une arborescence
riche mais néanmoins intuitive pour construire des graphiques
(objets `plotly.graph_objects.Figure`) pouvant être modifiés *a posteriori*
si besoin (par exemple pour *customiser* les axes). 

### Comment visualiser un graphique `plotly` ?

Dans un notebook Jupyter classique, les lignes suivantes de code permettent
d'afficher le résultat d'une commande `Plotly` sous un bloc de code: 

In [ ]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

Pour `JupyterLab`, l'extension `jupyterlab-plotly` s'avère nécessaire:

In [ ]:
jupyter labextension install jupyterlab-plotly

Pour les utilisateurs de `python` via l'excellent package `R` `reticulate`, il
est possible d'écrire le résultats dans un fichier `.html` et d'utiliser
`htmltools::includeHTML` pour l'afficher via `R Markdown` (les utilisateurs
de `R` trouveront bien-sûr une technique bien plus simple: utiliser
directement le package `R` `plotly`...)

### Réplication de l'exemple précédent avec `plotly`

Les modules suivant seront nécessaires pour construire des graphiques
avec `plotly`:

In [ ]:
import plotly
import plotly.express as px

**Exercice 4: le premier graphique avec plotly**

L'objectif est de reconstuire le premier diagramme en barre avec `plotly`

1. Utiliser la fonction adéquate avec `Plotly Express`. Ne pas prendre le
thème par défaut mais un à fond blanc, pour avoir un résultat ressemblant
à celui proposé sur le site de l'*open-data*. Pour la couleur rouge,
vous pouvez utiliser l'argument `color_discrete_sequence`. 
2. Ne pas oublier de nommer les axes, et la couleur du texte de l'axe inférieur
3. Tester un autre thème, à fond sombre. Pour les couleurs, faire un 
groupe stockant les trois plus fortes valeurs puis les autres.

In [ ]:
fig = px.bar(df1.sort_values('Comptage horaire', ascending = True), orientation = 'h', x='Comptage horaire', y='Nom du compteur', color_discrete_sequence=["red"], template = "plotly_white")
fig.update_layout(
    title='Les 10 compteurs avec la moyenne horaire la plus élevée',
    xaxis_title='Moyenne du comptage par heure sur la période sélectionnée')
fig.update_xaxes(title_font=dict(color='red'))

# Pour inclusion dans le site web
htmlsnip = plotly.io.to_html(fig, include_plotlyjs=False)

In [ ]:
df1['top'] = df1['Comptage horaire'] > df1.sort_values('Comptage horaire', ascending = False)['Comptage horaire'][3]
fig2 = px.bar(df1.sort_values('Comptage horaire', ascending = True), orientation = 'h', x='Comptage horaire', y='Nom du compteur', template = "plotly_dark", color = 'top')
fig2.update_layout(
    title='Les 10 compteurs avec la moyenne horaire la plus élevée',
    xaxis_title='Moyenne du comptage par heure sur la période sélectionnée')

# Pour inclusion dans le site web
htmlsnip2 = plotly.io.to_html(fig2, include_plotlyjs=False)

Le résultat devrait ressembler aux deux graphiques suivants:

{{< rawhtml >}}
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> 

In [ ]:
%%R
tablelight::print_html(py$htmlsnip)

{{< /rawhtml >}}

{{< rawhtml >}}
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> 

In [ ]:
%%R
tablelight::print_html(py$htmlsnip2)

{{< /rawhtml >}}